# Gloved vs Ungloved Hand Detection

This notebook loads my trained YOLOv8 model and runs hand detection on images from my Part_1_Glove_Detection project. Detected hands get boxed and labeled in saved images, and results are logged as JSON.


In [10]:
# Imports
from ultralytics import YOLO
import os
import json
from glob import glob
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import random


### Paths

All folders are already set up:
- Trained weights file: `best.pt` (placed in this same directory)
- Test images folder: `test_images` (test .jpg images here)
- Annotated images will be saved to `output/`
- JSON logs go into `logs/`


### Dataset and Model

Dataset: Gloves and bare hands detection from Roboflow Universe.
Classes: gloved_hand, bare_hand
Model: YOLOv8n pretrained on COCO, fine-tuned here on the glove dataset for 3 epochs.


# Train the model

In [11]:
# 1. Train the YOLOv8 model for glove vs bare hand detection
model = YOLO("yolov8n.pt")  # Load pretrained YOLOv8n

# Train on dataset YAML for 3 epochs (quick test)
results = model.train(
    data="../Gloves and bare hands detection.v2i.yolov8/data.yaml",  
    epochs=3,  
    imgsz=640
)


New https://pypi.org/project/ultralytics/8.3.184 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.182  Python-3.11.4 torch-2.8.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../Gloves and bare hands detection.v2i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train15, nb

train: Scanning C:\Users\Shreya Sharma\Desktop\Glove Ungloved hand detection\Submission\Gloves and bare hands detection

val: Fast image access  (ping: 0.10.0 ms, read: 1.90.7 MB/s, size: 15.0 KB)



C:\Users\Shreya Sharma\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\Shreya Sharma\Desktop\Glove Ungloved hand detection\Submission\Gloves and bare hands detection.v

Plotting labels to runs\detect\train15\labels.jpg... 


...

        3/3         0G      1.038       1.63      1.443         20        640: 100%|██████████| 45/45 [03:41<00:00,  4.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:15<0

                   all        139        181      0.526      0.576      0.529      0.316



3 epochs completed in 0.207 hours.
Optimizer stripped from runs\detect\train15\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train15\weights\best.pt, 6.2MB

Validating runs\detect\train15\weights\best.pt...
Ultralytics 8.3.182  Python-3.11.4 torch-2.8.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:14<0


                   all        139        181      0.529      0.576       0.53      0.317
         gloverotation         40         44      0.278      0.591      0.372      0.194
       surgical-gloves         99        137      0.781      0.562      0.687       0.44
Speed: 1.8ms preprocess, 85.5ms inference, 0.0ms loss, 6.0ms postprocess per image
Results saved to runs\detect\train15


## Detection Function: Run inference on a single image and save annotation


In [21]:
def run_detection(model, image_path, conf_thresh=0.25):
    img = cv2.imread(image_path)
    results = model(img, conf=conf_thresh)  # returns list of Results objects
    detections = []
    annotated_img = img.copy()
    names = model.names  # class names from the model

    for result in results:
        boxes = result.boxes
        if boxes is None or len(boxes) == 0:
            continue
        for box in boxes:
            cls = int(box.cls[0].item())
            label = names[cls]
            conf = float(box.conf[0].item())
            x1, y1, x2, y2 = [int(coord) for coord in box.xyxy[0].tolist()]

            color = (0, 255, 0) if "glove" in label.lower() else (0, 0, 255)
            cv2.rectangle(annotated_img, (x1, y1), (x2, y2), color, 2)
            cv2.putText(annotated_img, f"{label} {conf:.2f}", (x1, max(0, y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            detections.append({
                "label": label,
                "confidence": conf,
                "bbox": [x1, y1, x2, y2]
            })
    return annotated_img, detections


## Batch process folder: Run detection on all images in input folder,
## save annotated images in output folder, and JSON logs in logs folder.


In [22]:
def process_folder(model, input_folder="test_images", output_folder="output", logs_folder="logs", conf_thresh=0.25):
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(logs_folder, exist_ok=True)

    image_files = sorted(glob(os.path.join(input_folder, "*.jpg")))
    if not image_files:
        print(f"No .jpg images found in {input_folder}")
        return

    for img_path in image_files:
        annotated_img, detections = run_detection(model, img_path, conf_thresh)

        # Save annotated image
        out_img_path = os.path.join(output_folder, os.path.basename(img_path))
        cv2.imwrite(out_img_path, annotated_img)

        # Save JSON log
        log_data = {
            "filename": os.path.basename(img_path),
            "detections": detections
        }
        json_path = os.path.join(logs_folder, Path(img_path).stem + ".json")
        with open(json_path, "w") as jf:
            json.dump(log_data, jf, indent=2)

        print(f"Processed {os.path.basename(img_path)} - {len(detections)} detections")


## Run batch detection on test_images folder


In [23]:
process_folder(model, input_folder="test_images", output_folder="output", logs_folder="logs", conf_thresh=0.3)


0: 640x640 1 paper, 84.0ms
Speed: 3.8ms preprocess, 84.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Processed 1596291826438_jpg.rf.1028ec0fcc1a82906942bf201b2fd24c.jpg - 1 detections

0: 640x640 1 gloverotation, 73.4ms
Speed: 6.1ms preprocess, 73.4ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)
Processed 1596387616003_jpg.rf.1f48301f49cdd52717d572490d80e0f3.jpg - 1 detections

0: 640x640 3 gloverotations, 1 paper, 75.3ms
Speed: 4.6ms preprocess, 75.3ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)
Processed 1596466878995_jpg.rf.d2c5c10c8ca332d8658dd1aef88d486e.jpg - 4 detections

0: 640x640 2 gloverotations, 79.9ms
Speed: 5.8ms preprocess, 79.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Processed 1596467241185_jpg.rf.8a4877f5b00133383798120916a14758.jpg - 2 detections

0: 640x640 (no detections), 76.5ms
Speed: 5.5ms preprocess, 76.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


## Visualize one sample annotated image from output folder


In [25]:
import random
import matplotlib.pyplot as plt
output_images = os.listdir("output")
if output_images:
    sample_img = random.choice(output_images)
    img = cv2.cvtColor(cv2.imread(os.path.join("output", sample_img)), cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Sample annotated image: {sample_img}")
    plt.show()
else:
    print("No images found in output/ folder.")


<Figure size 800x600 with 1 Axes>

## Notes

- Trained for 3 epochs to quickly verify model’s ability to detect glove vs bare hands.
- Adjust confidence threshold in `process_folder()` for tuning detections.
- Organize folders as:
  - `test_images/` — input images
  - `output/` — annotated images saved here
  - `logs/` — JSON detection results saved here
- Use relative paths for portability.
